In [1]:
import datetime
import pandas as pd
import sys

sys.path.append("/home/sergey/drclinics/reports")
sys.path.append("/home/sergey/drclinics/common")

from log import log
from utils import get_path, DATETIME_FORMAT
from report_utils import save_to_excel, send_mail
from connect_platform import connect_telemed

In [2]:
sql_sub = """
select 	prod.full_name product_name,
		prod.amount,
		patient.id patient_id,
		person.first_name,
		person.middle_name,
		person.last_name,
		par."name" partner,
		p.start_ + 3 * interval '1 hour' sub_start,
		p.end_ + 3 * interval '1 hour' sub_end
from promotion p
inner join product_condition pc
   on p.product_condition_id = pc.id
inner join product prod
   on prod.id = pc.product_id
---
inner join patient_promotion pp on p.id = pp.promotion_id 
inner join patient on pp.patient_id = patient.id 
inner join person on patient.person_id = person.id 
---
inner join partner par on p.partner_id = par.id
where lower(prod.full_name) like '%подписка%' 
and first_name not like '%test%'
"""

In [3]:
sql_app = """

select 	prod.full_name product_name,
		prod.amount,
		patient.id patient_id,
		person.first_name,
		person.middle_name,
		person.last_name,
		par."name" partner,
		p.start_ + 3 * interval '1 hour' sub_start,
		p.end_ + 3 * interval '1 hour' sub_end,
		a.id,
		a.finished + 3 * interval '1 hour' app_finished
from promotion p
inner join product_condition pc
   on p.product_condition_id = pc.id
inner join product prod
   on prod.id = pc.product_id
---
inner join patient_promotion pp on p.id = pp.promotion_id 
inner join patient on pp.patient_id = patient.id 
inner join person on patient.person_id = person.id 
---
inner join partner par on p.partner_id = par.id
---
full join appointment a on patient.id = a.patient_id
where lower(prod.full_name) like '%подписка%' 
and a.finished > p.start_
and a.finished < p.end_
and first_name not like '%test%'
and a.good

"""

In [4]:
conn = connect_telemed()

2020-04-29 14:31:33 connect to postgres database using config file "/home/sergey/drclinics/common/../.credentials/telemed/prom.cfg"
2020-04-29 14:31:33 creating ssh tunnel to 172.16.100.19 as root...
2020-04-29 14:31:33 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 37271
2020-04-29 14:31:33 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [5]:
df: pd.DataFrame = conn.query(sql_sub)

2020-04-29 14:31:33 @telemed query:
                    select 	prod.full_name product_name,
				    		prod.amount,
				    		patient.id patient_id,
				    		person.first_name,
				    		person.middle_name,
				    		person.last_name,
				    		par."name" partner,
				    		p.start_ + 3 * interval '1 hour' sub_start,
				    		p.end_ + 3 * interval '1 hour' sub_end
				    from promotion p
				    inner join product_condition pc
				       on p.product_condition_id = pc.id
				    inner join product prod
				       on prod.id = pc.product_id
				    ---
				    inner join patient_promotion pp on p.id = pp.promotion_id 
				    inner join patient on pp.patient_id = patient.id 
				    inner join person on patient.person_id = person.id 
				    ---
				    inner join partner par on p.partner_id = par.id
				    where lower(prod.full_name) like '%подписка%' 
				    and first_name not like '%test%'
                    None
2020-04-29 14:31:33 return 881 rows


In [6]:
df_app: pd.DataFrame = conn.query(sql_app)

2020-04-29 14:31:34 @telemed query:
                    select 	prod.full_name product_name,
				    		prod.amount,
				    		patient.id patient_id,
				    		person.first_name,
				    		person.middle_name,
				    		person.last_name,
				    		par."name" partner,
				    		p.start_ + 3 * interval '1 hour' sub_start,
				    		p.end_ + 3 * interval '1 hour' sub_end,
				    		a.id,
				    		a.finished + 3 * interval '1 hour' app_finished
				    from promotion p
				    inner join product_condition pc
				       on p.product_condition_id = pc.id
				    inner join product prod
				       on prod.id = pc.product_id
				    ---
				    inner join patient_promotion pp on p.id = pp.promotion_id 
				    inner join patient on pp.patient_id = patient.id 
				    inner join person on patient.person_id = person.id 
				    ---
				    inner join partner par on p.partner_id = par.id
				    ---
				    full join appointment a on patient.id = a.patient_id
				    where lower(prod.full_name) li

In [7]:
conn.close()

2020-04-29 14:31:35 @telemed: closing postgres connection...
2020-04-29 14:31:35 closing ssh tunnel to 172.16.100.19...


In [8]:
#df = pd.read_csv('par-pro_202004271813.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   product_name  881 non-null    object        
 1   amount        881 non-null    object        
 2   patient_id    881 non-null    int64         
 3   first_name    881 non-null    object        
 4   middle_name   777 non-null    object        
 5   last_name     881 non-null    object        
 6   partner       881 non-null    object        
 7   sub_start     881 non-null    datetime64[ns]
 8   sub_end       881 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 62.1+ KB


In [10]:
df.partner.unique()

array(['Доктор рядом', 'Яндекс.Еда', 'Альфа Страхование'], dtype=object)

In [11]:
df.head()

,product_name,amount,patient_id,first_name,middle_name,last_name,partner,sub_start,sub_end
0,Подписка на 1 месяц. Личная клиника. Семейный.490,490.00,3172903,Николай,Юрьевич,Загуляев,Доктор рядом,2020-04-08 03:00:00,2020-05-08 03:00:00
1,Подписка на 1 месяц. Детский. 150,150.00,2211623,Нина,Максимовна,Гладышева,Доктор рядом,2020-04-03 03:00:00,2020-05-03 03:00:00
2,Подписка на 1 месяц. Детский. 290,290.00,2916927,Ирина,Вадимовна,Иванова,Доктор рядом,2020-03-06 03:00:00,2020-05-09 03:00:00
3,Подписка ТМ ВВ,2400.00,3046788,Test,Test,Test,Доктор рядом,2020-04-03 03:00:00,2020-04-10 03:00:00
4,Подписка ТМ ВВ,2400.00,3046788,Test,Test,Test,Доктор рядом,2020-04-03 03:00:00,2020-04-10 03:00:00


In [12]:
df[df['partner'] == 'Яндекс.Еда']

,product_name,amount,patient_id,first_name,middle_name,last_name,partner,sub_start,sub_end
400,Яндекс.Еда_Подписка,25.00,2583249,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
430,Яндекс.Еда_Подписка,25.00,2583249,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
450,Яндекс.Еда_Подписка,25.00,2583206,Ксения,Марсовна,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
451,Яндекс.Еда_Подписка,25.00,2583219,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
605,Яндекс.Еда_Подписка,25.00,2583219,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
606,Яндекс.Еда_Подписка,25.00,2583206,Ксения,Марсовна,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00


In [13]:
df[df['product_name'] == 'Яндекс.Еда_Подписка']

,product_name,amount,patient_id,first_name,middle_name,last_name,partner,sub_start,sub_end
400,Яндекс.Еда_Подписка,25.00,2583249,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
430,Яндекс.Еда_Подписка,25.00,2583249,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
450,Яндекс.Еда_Подписка,25.00,2583206,Ксения,Марсовна,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
451,Яндекс.Еда_Подписка,25.00,2583219,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
605,Яндекс.Еда_Подписка,25.00,2583219,Игорь,Марсович,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00
606,Яндекс.Еда_Подписка,25.00,2583206,Ксения,Марсовна,Гаврилюк,Яндекс.Еда,2020-02-12 03:00:00,2021-02-11 03:00:00


In [14]:
df.groupby(['product_name', 'partner'])[['patient_id']].count().reset_index()

,product_name,partner,patient_id
0,Годовая подписка – базовый_1800,Доктор рядом,8
1,Подписка ТМ ВВ,Доктор рядом,30
2,Подписка ТМ ВВ мес.,Доктор рядом,34
3,Подписка ТМ ВВ.Довериум мес.,Доктор рядом,18
4,Подписка ТМ ДР Семейная 1 мес,Доктор рядом,2
5,Подписка на 1 месяц. Базовый,Доктор рядом,42
6,Подписка на 1 месяц. Детский. 150,Доктор рядом,404
7,Подписка на 1 месяц. Детский. 190,Доктор рядом,96
8,Подписка на 1 месяц. Детский. 290,Доктор рядом,80
9,Подписка на 1 месяц. Детский. 390,Доктор рядом,1


In [16]:
df.groupby(['partner'])[['patient_id']].count().reset_index().rename(columns={'patient_id':'total_patients'})

,partner,total_patients
0,Альфа Страхование,5
1,Доктор рядом,870
2,Яндекс.Еда,6


In [17]:
df_subscribe = pd.DataFrame(df.groupby(['partner', 'product_name'])[['patient_id']].count()).reset_index()

In [18]:
df_subscribe = df_subscribe.rename(columns={'patient_id':'subscribe'})

In [19]:
df_subscribe # всего подписок

,partner,product_name,subscribe
0,Альфа Страхование,Телемедицина_подписка_АС_490,1
1,Альфа Страхование,Телемедицина_подписка_АС_690,2
2,Альфа Страхование,Телемедицина_подписка_АС_990,2
3,Доктор рядом,Годовая подписка – базовый_1800,8
4,Доктор рядом,Подписка ТМ ВВ,30
5,Доктор рядом,Подписка ТМ ВВ мес.,34
6,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,18
7,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,2
8,Доктор рядом,Подписка на 1 месяц. Базовый,42
9,Доктор рядом,Подписка на 1 месяц. Детский. 150,404


In [20]:
df_temp = pd.DataFrame(df)

In [21]:
df_temp = df_temp.groupby(['partner', 'product_name'])['patient_id'].unique().reset_index()

In [22]:
df_temp

,partner,product_name,patient_id
0,Альфа Страхование,Телемедицина_подписка_АС_490,[3000092]
1,Альфа Страхование,Телемедицина_подписка_АС_690,[3000092]
2,Альфа Страхование,Телемедицина_подписка_АС_990,[3000092]
3,Доктор рядом,Годовая подписка – базовый_1800,"[3046858, 1118833, 2600071, 955694, 4386]"
4,Доктор рядом,Подписка ТМ ВВ,"[3046788, 1118833, 955694, 3191625, 3514719, 1..."
5,Доктор рядом,Подписка ТМ ВВ мес.,"[3052726, 3052608, 1045350, 3052652, 3052586, ..."
6,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,"[3484736, 77761, 3494396, 3508587, 3510464, 11..."
7,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,[1118833]
8,Доктор рядом,Подписка на 1 месяц. Базовый,"[1642619, 1118833, 955694, 1479649, 1190533, 1..."
9,Доктор рядом,Подписка на 1 месяц. Детский. 150,"[2211623, 2510009, 2917009, 2927792, 2567220, ..."


In [23]:
df_temp['prolong'] = ''

for i in range(df_temp.shape[0]):
    df_temp['prolong'][i] = len(df_temp['patient_id'][i])

In [24]:
df_temp

,partner,product_name,patient_id,prolong
0,Альфа Страхование,Телемедицина_подписка_АС_490,[3000092],1
1,Альфа Страхование,Телемедицина_подписка_АС_690,[3000092],1
2,Альфа Страхование,Телемедицина_подписка_АС_990,[3000092],1
3,Доктор рядом,Годовая подписка – базовый_1800,"[3046858, 1118833, 2600071, 955694, 4386]",5
4,Доктор рядом,Подписка ТМ ВВ,"[3046788, 1118833, 955694, 3191625, 3514719, 1...",9
5,Доктор рядом,Подписка ТМ ВВ мес.,"[3052726, 3052608, 1045350, 3052652, 3052586, ...",14
6,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,"[3484736, 77761, 3494396, 3508587, 3510464, 11...",9
7,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,[1118833],1
8,Доктор рядом,Подписка на 1 месяц. Базовый,"[1642619, 1118833, 955694, 1479649, 1190533, 1...",26
9,Доктор рядом,Подписка на 1 месяц. Детский. 150,"[2211623, 2510009, 2917009, 2927792, 2567220, ...",366


In [25]:
df_subscribe['unique_users'] = ''

for i in range(df_temp.shape[0]):
    df_subscribe['unique_users'][i] = len(df_temp['patient_id'][i])

In [26]:
df_subscribe['prolongation'] = df_subscribe['subscribe'] - df_subscribe['unique_users']

In [27]:
df_subscribe

,partner,product_name,subscribe,unique_users,prolongation
0,Альфа Страхование,Телемедицина_подписка_АС_490,1,1,0
1,Альфа Страхование,Телемедицина_подписка_АС_690,2,1,1
2,Альфа Страхование,Телемедицина_подписка_АС_990,2,1,1
3,Доктор рядом,Годовая подписка – базовый_1800,8,5,3
4,Доктор рядом,Подписка ТМ ВВ,30,9,21
5,Доктор рядом,Подписка ТМ ВВ мес.,34,14,20
6,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,18,9,9
7,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,2,1,1
8,Доктор рядом,Подписка на 1 месяц. Базовый,42,26,16
9,Доктор рядом,Подписка на 1 месяц. Детский. 150,404,366,38


In [28]:
df_sum = df.groupby(['partner', 'product_name'])[['amount']].sum().reset_index()

In [29]:
df_sum

,partner,product_name,amount
0,Альфа Страхование,Телемедицина_подписка_АС_490,490.00
1,Альфа Страхование,Телемедицина_подписка_АС_690,1380.00
2,Альфа Страхование,Телемедицина_подписка_АС_990,1980.00
3,Доктор рядом,Годовая подписка – базовый_1800,14400.00
4,Доктор рядом,Подписка ТМ ВВ,72000.00
5,Доктор рядом,Подписка ТМ ВВ мес.,10200.00
6,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,8820.00
7,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,980.00
8,Доктор рядом,Подписка на 1 месяц. Базовый,6300.00
9,Доктор рядом,Подписка на 1 месяц. Детский. 150,60600.00


In [30]:
df_subscribe['total_revenue'] = ''

for i in range(df_temp.shape[0]):
    df_subscribe['total_revenue'][i] = df_sum['amount'][i]

In [31]:
df_subscribe

,partner,product_name,subscribe,unique_users,prolongation,total_revenue
0,Альфа Страхование,Телемедицина_подписка_АС_490,1,1,0,490.00
1,Альфа Страхование,Телемедицина_подписка_АС_690,2,1,1,1380.00
2,Альфа Страхование,Телемедицина_подписка_АС_990,2,1,1,1980.00
3,Доктор рядом,Годовая подписка – базовый_1800,8,5,3,14400.00
4,Доктор рядом,Подписка ТМ ВВ,30,9,21,72000.00
5,Доктор рядом,Подписка ТМ ВВ мес.,34,14,20,10200.00
6,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,18,9,9,8820.00
7,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,2,1,1,980.00
8,Доктор рядом,Подписка на 1 месяц. Базовый,42,26,16,6300.00
9,Доктор рядом,Подписка на 1 месяц. Детский. 150,404,366,38,60600.00


In [37]:
df_apps_temp = df_app.groupby(['partner', 'product_name'])['patient_id'].count().reset_index().rename(columns={'patient_id':'apps'})

In [38]:
df_apps_temp

,partner,product_name,apps
0,Доктор рядом,Годовая подписка – базовый_1800,23
1,Доктор рядом,Подписка ТМ ВВ,2
2,Доктор рядом,Подписка ТМ ВВ мес.,10
3,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,40
4,Доктор рядом,Подписка на 1 месяц. Базовый,34
5,Доктор рядом,Подписка на 1 месяц. Детский. 150,1617
6,Доктор рядом,Подписка на 1 месяц. Детский. 190,339
7,Доктор рядом,Подписка на 1 месяц. Детский. 290,255
8,Доктор рядом,Подписка на 1 месяц. Личная клиника. Семейный.490,143
9,Доктор рядом,Подписка на 1 месяц. Расширенный,20


In [44]:
df_subscribe['apps'] = ''

for i in range(df_subscribe.shape[0]):
    for j in range(df_apps_temp.shape[0]):
        if df_apps_temp['partner'][j] == df_subscribe['partner'][i] and df_apps_temp['product_name'][j] == df_subscribe['product_name'][i]:
            df_subscribe['apps'][i] = df_apps_temp['apps'][j]
            break
    else:
        df_subscribe['apps'][i] = 0
    

In [46]:
df_subscribe

,partner,product_name,subscribe,unique_users,prolongation,total_revenue,apps
0,Альфа Страхование,Телемедицина_подписка_АС_490,1,1,0,490.00,0
1,Альфа Страхование,Телемедицина_подписка_АС_690,2,1,1,1380.00,0
2,Альфа Страхование,Телемедицина_подписка_АС_990,2,1,1,1980.00,0
3,Доктор рядом,Годовая подписка – базовый_1800,8,5,3,14400.00,23
4,Доктор рядом,Подписка ТМ ВВ,30,9,21,72000.00,2
5,Доктор рядом,Подписка ТМ ВВ мес.,34,14,20,10200.00,10
6,Доктор рядом,Подписка ТМ ВВ.Довериум мес.,18,9,9,8820.00,40
7,Доктор рядом,Подписка ТМ ДР Семейная 1 мес,2,1,1,980.00,0
8,Доктор рядом,Подписка на 1 месяц. Базовый,42,26,16,6300.00,34
9,Доктор рядом,Подписка на 1 месяц. Детский. 150,404,366,38,60600.00,1617
